In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826948e+12,  1.05165000e+05,  1.04909800e+05, ...,
         1.04768000e+05,  4.98372300e+03, -1.00000000e+00],
       [ 1.73827092e+12,  1.04909800e+05,  1.05189300e+05, ...,
         1.04737000e+05,  4.83362100e+03, -1.00000000e+00],
       [ 1.73827926e+12,  1.05189200e+05,  1.04416800e+05, ...,
         1.04356100e+05,  4.86977900e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([21520, 22983]))
(array([0, 1]), array([4240, 4661]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-19 16:32:24,979] A new study created in memory with name: no-name-c5ce0601-7694-451e-b1ce-049f0e9bb881
[I 2025-03-19 16:32:50,908] Trial 0 finished with value: 0.8758695700574416 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 248, 'max_depth': 92, 'min_gain_to_split': 0.9401370984962135, 'min_data_in_leaf': 200, 'lambda_l1': 91.20517481461545, 'lambda_l2': 24.455655177299022, 'num_boost_round': 1990}. Best is trial 0 with value: 0.8758695700574416.


precision: 0.7898116438356164, recall: 0.791675606093113, f1: 0.7907425265188043


[I 2025-03-19 16:33:11,005] Trial 1 finished with value: 0.8752288155833432 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 70, 'max_depth': 610, 'min_gain_to_split': 0.9429108092494416, 'min_data_in_leaf': 439, 'lambda_l1': 98.46028545677399, 'lambda_l2': 40.17306511500757, 'num_boost_round': 662}. Best is trial 0 with value: 0.8758695700574416.


precision: 0.7899484536082474, recall: 0.7891010512765501, f1: 0.7895245250617151


[I 2025-03-19 16:34:10,110] Trial 2 finished with value: 0.8772596171361722 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 239, 'max_depth': 468, 'min_gain_to_split': 0.8719570736487325, 'min_data_in_leaf': 170, 'lambda_l1': 75.41919291452923, 'lambda_l2': 78.49790806651801, 'num_boost_round': 1646}. Best is trial 2 with value: 0.8772596171361722.


...

precision: 0.7895986189037548, recall: 0.7850246728169921, f1: 0.7873050026896181


[I 2025-03-19 17:14:27,873] Trial 98 finished with value: 0.8807007565790806 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 194, 'max_depth': 339, 'min_gain_to_split': 0.5500772199614717, 'min_data_in_leaf': 40, 'lambda_l1': 27.61401392367179, 'lambda_l2': 0.30175961120398886, 'num_boost_round': 769}. Best is trial 46 with value: 0.8827414758352123.


precision: 0.7961123110151188, recall: 0.7908174211542588, f1: 0.7934560327198364


[I 2025-03-19 17:14:52,742] Trial 99 finished with value: 0.8817000663878916 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 187, 'max_depth': 450, 'min_gain_to_split': 0.7088802613388953, 'min_data_in_leaf': 20, 'lambda_l1': 9.464513760703472, 'lambda_l2': 37.81610969285717, 'num_boost_round': 643}. Best is trial 46 with value: 0.8827414758352123.


precision: 0.8016978667827601, recall: 0.790173782450118, f1: 0.7958941112911939


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 277,
 'max_depth': 82,
 'min_gain_to_split': 0.3633515719216471,
 'min_data_in_leaf': 226,
 'lambda_l1': 12.10788430511262,
 'lambda_l2': 39.6412374402436,
 'num_boost_round': 538}

In [7]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8827414758352123


In [10]:
model.save_model("custom_indicators/models/model_side.txt")
prod_model.save_model("custom_indicators/models/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_1h.parquet")
meta_label = np.load("data/label_meta_1h.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_comb_spectrum_pwr_14         0
1h_comb_spectrum_pwr_13         0
1h_comb_spectrum_pwr_12         0
1h_comb_spectrum_pwr_11         0
side_model_res                  0
Length: 11842, dtype: int64

In [2]:
df_features = df_features[364:]
meta_label = meta_label[364:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(44503, 11842)
(44503, 7)


In [3]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([ 5682, 29920]))
(array([0, 1]), array([1873, 7028]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.25, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-19 23:29:16,165] A new study created in memory with name: no-name-88d1a4dd-8c4b-4ba3-99a5-ba94fb88ac19
[I 2025-03-19 23:29:42,298] Trial 0 finished with value: 0.7131413263755055 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 195, 'max_depth': 157, 'min_gain_to_split': 0.8217533883408678, 'min_data_in_leaf': 289, 'lambda_l1': 89.97360671415655, 'lambda_l2': 34.02503998553866, 'num_boost_round': 608}. Best is trial 0 with value: 0.7131413263755055.


precision: 0.899741506646972, recall: 0.6933693796243597, f1: 0.7831886853101897, auc: 0.7738605489566409, fbeta: 0.7131413263755055


[I 2025-03-19 23:30:04,286] Trial 1 finished with value: 0.7410333904876761 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 367, 'max_depth': 814, 'min_gain_to_split': 0.7417348217797284, 'min_data_in_leaf': 272, 'lambda_l1': 36.71073766350226, 'lambda_l2': 62.33331128899878, 'num_boost_round': 1201}. Best is trial 1 with value: 0.7410333904876761.


precision: 0.8709884467265725, recall: 0.7723392145702903, f1: 0.8187028657616893, auc: 0.7732577431863576, fbeta: 0.7410333904876761


[I 2025-03-19 23:30:40,523] Trial 2 finished with value: 0.74378967407658 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 441, 'max_depth': 509, 'min_gain_to_split': 0.40774568310909015, 'min_data_in_leaf': 240, 'lambda_l1': 30.87540960847114, 'lambda_l2': 0.8902552705564973, 'num_boost_round': 576}. Best is trial 2 with value: 0.74378967407658.


...

precision: 0.8522778192681105, recall: 0.8118952760387024, f1: 0.8315965896669825, auc: 0.7684406907493206, fbeta: 0.745999393231011


[I 2025-03-20 02:02:29,228] Trial 98 finished with value: 0.7505231797096819 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 300, 'max_depth': 69, 'min_gain_to_split': 0.3234112371768401, 'min_data_in_leaf': 169, 'lambda_l1': 7.8957857357188255, 'lambda_l2': 75.14223459296814, 'num_boost_round': 1301}. Best is trial 81 with value: 0.7572403994124306.


precision: 0.8412540785927082, recall: 0.8437677859988617, f1: 0.8425090573275555, auc: 0.7700404240713905, fbeta: 0.7505231797096819


[I 2025-03-20 02:04:45,047] Trial 99 finished with value: 0.7542662563844025 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 354, 'max_depth': 271, 'min_gain_to_split': 0.3718532116987835, 'min_data_in_leaf': 238, 'lambda_l1': 3.1037393348116353, 'lambda_l2': 70.77598352744104, 'num_boost_round': 1243}. Best is trial 81 with value: 0.7572403994124306.


precision: 0.8430290162774239, recall: 0.8474672737620945, f1: 0.8452423188817143, auc: 0.7721974583551235, fbeta: 0.7542662563844025


In [5]:
study.best_params

{'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 316,
 'max_depth': 275,
 'min_gain_to_split': 0.03631507188972556,
 'min_data_in_leaf': 133,
 'lambda_l1': 14.268708952393501,
 'lambda_l2': 71.66235509706269,
 'num_boost_round': 1335}

In [6]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("custom_indicators/models/model_meta.txt")
prod_model.save_model("custom_indicators/models/model_meta_prod.txt")